In [235]:
f = open("Native2ASCIIEncoding.txt", "r")
UAST_text = f.readlines()
f.close()

In [236]:
import re
tokens = []
for line in UAST_text :
    lineTokens = line.split()
    tokens = tokens + lineTokens

In [237]:
#for token in tokens :
    #print(token)

In [238]:
type_stack = []
general_stack = []
bracket_stack = []
result = []
class_bracket_count = 0
method_bracket_count = 0
class_name_flag = 1
method_name_flag = 1

In [239]:
for i in range(0, len(tokens)) :
    #print(type_stack)
    token = tokens[i]
    if(token == "{") :
        bracket_stack = bracket_stack + ["{"]
        if(len(type_stack) == 1) :
            class_bracket_count = class_bracket_count + 1
        elif(len(type_stack) == 2) :
            method_bracket_count = method_bracket_count + 1
    if(i+2 < (len(tokens)-1)) :
        if(token == "{" and tokens[i+2] == "\"ClassOrInterfaceDeclaration\"") :
            type_stack = type_stack + ["ClassOrInterfaceDeclaration"]
            class_name_flag = 0               #name of class not yet known
            class_bracket_count = class_bracket_count + 1
    if(i+2 < (len(tokens)-1)) :
        if(token == "{" and tokens[i+2] == "\"MethodDeclaration\"") :
            type_stack = type_stack + ["MethodDeclaration"]
            method_name_flag = 0             #name of method not yet known
            method_bracket_count = method_bracket_count + 1
            class_bracket_count = class_bracket_count + 1
    if(token == "internalType:" and tokens[i+1] == "\"SimpleName\"") :
        if(len(type_stack) >= 2 and method_name_flag == 0) :
            method_name = tokens[i+17]
            method_name_flag = 1    #name of method known
            method_info = [class_name, method_name]
            #print(method_info)
        elif(len(type_stack) == 1 and class_name_flag == 0) :
            class_name = tokens[i+17]
            class_name_flag = 1           
    
    if(token == "token:" and class_name_flag == 1 and method_name_flag == 1 and len(type_stack) >= 2) :
        data = data + [tokens[i+1]]
           
    if(token == "}") :
        bracket_stack.pop()
        if(len(type_stack) >= 2) :
            method_bracket_count = method_bracket_count - 1
            class_bracket_count = class_bracket_count - 1
            if(method_bracket_count == 0) :
                type_stack.pop()
                method_data = method_info + data
                #print(method_data)
                result = result + [method_data]
                data = []
                method_info = []
        if(len(type_stack) == 1) :
            #print(class_bracket_count)
            class_bracket_count = class_bracket_count - 1
            if(class_bracket_count == 0) :
                type_stack.pop()

In [261]:
code_summary = []
for i in range(0, len(result)) :
    method_data = result[i]
    if (method_data == []) :
        continue
    class_name = method_data[0].replace("\"", "")
    method_name = method_data[1].replace("\"", "")
    method_summary = [class_name, method_name]
    method_words = []
    for j in range(2, len(method_data)) :
        token = method_data[j]
        token = token.replace("\"", "")
        if(len(token) >= 4 and ("\\" not in token ) and (":" not in token )) :
            method_words = method_words + [token]
    method_words = list(set(method_words))
    code_summary = code_summary + [method_summary + method_words]

In [264]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

for documents in code_summary :
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(documents)

    true_k = 1
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i),
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind])


Top terms per cluster:
Cluster 0:
 gettextreader
 reader
 native2asciireader
 native2asciiencoding
 ioexception
 inputstream
 false
Top terms per cluster:
Cluster 0:
 gettextwriter
 writer
 outputstream
 native2asciiencoding
 ioexception
 filterwriter
 asciiencoding
Top terms per cluster:
Cluster 0:
 append
 writer
 write
 tostring
 subsequence
 start
 null
 native2asciiencoding
 ioexception
 equals
Top terms per cluster:
Cluster 0:
 append
 writer
 write
 native2asciiencoding
 ioexception
 char
Top terms per cluster:
Cluster 0:
 write
 void
 string
 native2asciiencoding
 length
 ioexception
Top terms per cluster:
Cluster 0:
 write
 void
 native2asciiencoding
 length
 ioexception
 char
 cbuf
Top terms per cluster:
Cluster 0:
 write
 void
 tochararray
 substring
 string
 plus
 native2asciiencoding
 ioexception
Top terms per cluster:
Cluster 0:
 write
 void
 tochararray
 string
 postfix_increment
 plus
 native2asciiencoding
 multiply
 ioexception
 format
Top terms per cluster:
Cluster 0: